In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!unrar x "/content/drive/My Drive/songs_polaND.rar" "/content/drive/My Drive/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/songs_polaND.rar

Creating    /content/drive/My Drive/songs_polaND                      OK
Creating    /content/drive/My Drive/songs_polaND/Alaudaarvensis       OK
Extracting  /content/drive/My Drive/songs_polaND/Alaudaarvensis/Alaudaarvensis131537.wav       0%  OK 
Extracting  /content/drive/My Drive/songs_polaND/Alaudaarvensis/Alaudaarvensis131690.wav       0%  OK 
Extracting  /content/drive/My Drive/songs_polaND/Alaudaarvensis/Alaudaarvensis131692.wav       0%  OK 
Extracting  /content/drive/My Drive/songs_polaND/Alaudaarvensis/Alaudaarvensis138225.wav       0%  OK 
Extracting  /content/drive/My Drive/songs_polaND/Alaudaarvensis/Alaudaarvensis168330.wav       0%  OK 
Extracting  /content/drive/My Drive/songs_polaND/Alaudaarvensis/Alaudaarvensis176345.wav       0%  OK 
Extracting  /content/drive/My Drive/songs_polaND/Alauda

In [ ]:
import numpy as np
import pandas as pd
import librosa

In [ ]:
def zcr(audio, sr, frame, overlap):
    return librosa.feature.zero_crossing_rate(audio, frame_length=int((frame / 1000) * sr), hop_length=int((1 - overlap) * frame * sr / 1000))

In [ ]:
(audio, fs) = librosa.load('/content/drive/My Drive/songs_polaND/Turduspilaris/Turduspilaris565575.wav', duration = 5)

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(audio, rate = fs)

In [ ]:
x = zcr(audio, fs, 20, 0)

In [ ]:
x.shape

(1, 424)

In [ ]:
import os
import csv

In [ ]:
header = 'filename'
#for i in range(1, 301):
#    header += f' zcr{i}'
header += ' zcr_mean'
header += ' zcr_var'
header += ' label'
header = header.split()

In [ ]:
file = open('/content/drive/My Drive/zcr.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
#genres = 'belly_pain burping discomfort hungry tired'.split()
genres = os.listdir('/content/drive/My Drive/songs_polaND/')
for g in genres:
    for filename in os.listdir(f'/content/drive/My Drive/songs_polaND/{g}'):
        songname = f'/content/drive/My Drive/songs_polaND/{g}/{filename}'
        audio, fs = librosa.load(songname, duration=5)
        
        zcr_ = zcr(audio, fs, 20, 0)
        
        to_append = f'{filename} {np.mean(zcr_)} {np.var(zcr_)}'    
        #for e in zcr_[0]:
        #    to_append += f' {e}'
        to_append += f' {g}'
        file = open('/content/drive/My Drive/zcr.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
def spectral_rolloff(audio, sr, n_fft = 2048, overlap = 0.75):
    return librosa.feature.spectral_rolloff(audio, sr, n_fft=n_fft, hop_length=int((1 - overlap) * n_fft))

In [ ]:
header = 'filename'
#for i in range(1, 301):
#    header += f' zcr{i}'
header += ' sro_mean'
header += ' sro_var'
header += ' label'
header = header.split()

In [ ]:
file = open('/content/drive/My Drive/sro.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
#genres = 'belly_pain burping discomfort hungry tired'.split()
genres = os.listdir('/content/drive/My Drive/songs_polaND/')
for g in genres:
    for filename in os.listdir(f'/content/drive/My Drive/songs_polaND/{g}'):
        songname = f'/content/drive/My Drive/songs_polaND/{g}/{filename}'
        audio, fs = librosa.load(songname, duration=5)
        
        sro_ = spectral_rolloff(audio, fs)
        
        to_append = f'{filename} {np.mean(sro_)} {np.var(sro_)}'    
        #for e in sro_[0]:
        #    to_append += f' {e}'
        to_append += f' {g}'
        file = open('/content/drive/My Drive/sro.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

###MODELS

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("/content/drive/My Drive/zcr.csv")
df.head()

,filename,zcr_mean,zcr_var,label
0,Alaudaarvensis131537.wav,0.258612,0.011728,Alaudaarvensis
1,Alaudaarvensis131690.wav,0.337134,0.005628,Alaudaarvensis
2,Alaudaarvensis131692.wav,0.325079,0.006040,Alaudaarvensis
3,Alaudaarvensis138225.wav,0.172789,0.009284,Alaudaarvensis
4,Alaudaarvensis168330.wav,0.253741,0.009359,Alaudaarvensis


In [ ]:
df.drop('filename', axis=1, inplace=True)

In [ ]:
df.head()

,zcr_mean,zcr_var,label
0,0.258612,0.011728,Alaudaarvensis
1,0.337134,0.005628,Alaudaarvensis
2,0.325079,0.006040,Alaudaarvensis
3,0.172789,0.009284,Alaudaarvensis
4,0.253741,0.009359,Alaudaarvensis


In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
X.shape

(1404, 2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify=y)

scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test) 

models = []
models.append(('LR', LogisticRegression()))
models.append(('SVM', SVC()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('RF', RandomForestClassifier()))

In [ ]:
wt = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
classes = np.unique(y_train)
tmp = {}
for i in range(len(wt)):
    tmp[classes[i]] = wt[i]
swt = [tmp[_] for _ in y_train]

In [ ]:
print(len(swt))
print(y_train.shape)

1123
(1123,)


In [ ]:
for name, model in models:
    if(name != 'KNN'):
        model.fit(X_train, y_train, sample_weight=swt)
    else:
        model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Summary of the predictions made by the classifier
    print(name)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    

LR
                              precision    recall  f1-score   support

              Alaudaarvensis       0.00      0.00      0.00         7
                    Apusapus       0.00      0.00      0.00         1
          Cardueliscarduelis       0.00      0.00      0.00         5
              Chlorischloris       0.00      0.00      0.00         9
Coccothraustescoccothraustes       0.00      0.00      0.00         2
             Columbapalumbus       0.29      0.67      0.40         6
            Corvusfrugilegus       0.00      0.00      0.00         1
          Emberizacitrinella       0.27      0.81      0.41        37
           Erithacusrubecula       0.45      0.29      0.36        17
            Fringillacoelebs       0.00      0.00      0.00        23
          Garrulusglandarius       0.00      0.00      0.00         4
            Luscinialuscinia       0.20      0.21      0.21        19
               Motacillaalba       0.00      0.00      0.00         2
            Pass

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RF
                              precision    recall  f1-score   support

              Alaudaarvensis       0.00      0.00      0.00         7
                    Apusapus       0.00      0.00      0.00         1
          Cardueliscarduelis       0.20      0.20      0.20         5
              Chlorischloris       0.00      0.00      0.00         9
Coccothraustescoccothraustes       0.00      0.00      0.00         2
             Columbapalumbus       0.43      0.50      0.46         6
            Corvusfrugilegus       0.00      0.00      0.00         1
          Emberizacitrinella       0.33      0.51      0.40        37
           Erithacusrubecula       0.25      0.18      0.21        17
            Fringillacoelebs       0.09      0.13      0.11        23
          Garrulusglandarius       0.00      0.00      0.00         4
            Luscinialuscinia       0.19      0.16      0.17        19
               Motacillaalba       0.00      0.00      0.00         2
            Pass